In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
tf.__version__

'2.3.1'

In [4]:
data = pd.read_csv('Churn_Modelling.csv')
x = data.iloc[:,3:-1].values
y = data.iloc[:,-1].values

In [5]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [6]:
print(y)

[1 0 1 ... 1 1 0]


# Data Preprocessing

## Encoding the Gender Variable

In [7]:
# Mapping the categorical variables, we can also do mapping but new method, here we go!

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [8]:
print(x) # it encoded Female to 0 and male to 1, the mapping purely machine based!

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


## OneHotEncoding the Geography Variable

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [10]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [11]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=143, test_size=0.2, shuffle=True)

## Feature Scaling

In [12]:
# We will feature scale all the dummy variable too in DL

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# ANN

In [13]:
model = tf.keras.models.Sequential()

In [14]:
model.add(tf.keras.layers.Dense(units=6, activation='relu'))     #1st Hidden Layer
model.add(tf.keras.layers.Dense(units=6, activation='relu'))     #2nd Hidden Layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))  #Output layer

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(x_train_scaled, y_train, epochs=150, batch_size=32)

Epoch 1/150
250/250 [==============================] - 0s 328us/step - loss: 0.5612 - accuracy: 0.7446
Epoch 2/150
250/250 [==============================] - 0s 318us/step - loss: 0.4723 - accuracy: 0.7962
Epoch 3/150
250/250 [==============================] - 0s 326us/step - loss: 0.4404 - accuracy: 0.8101
Epoch 4/150
250/250 [==============================] - 0s 310us/step - loss: 0.4232 - accuracy: 0.8200
Epoch 5/150
250/250 [==============================] - 0s 308us/step - loss: 0.4100 - accuracy: 0.8289
Epoch 6/150
250/250 [==============================] - 0s 357us/step - loss: 0.3981 - accuracy: 0.8357
Epoch 7/150
250/250 [==============================] - 0s 403us/step - loss: 0.3875 - accuracy: 0.8411
Epoch 8/150
250/250 [==============================] - 0s 412us/step - loss: 0.3787 - accuracy: 0.8438
Epoch 9/150
250/250 [==============================] - 0s 320us/step - loss: 0.3711 - accuracy: 0.8459
Epoch 10/150
250/250 [==============================] - 0s 315us/step - l

## Predicting the single customer

In [17]:
# We must input the independent variables in the same order that we trained our model
# The inputs must be in scaled 2-D arrays


Pred = model.predict(scaler.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
print(Pred)


#This gives the probability of Yes to leaving bank, it is low than 0.5 the customer will stay!

[[0.02938107]]


## Test Set Results

In [18]:
y_pred = model.predict(x_test_scaled)
y_pred = (y_pred > 0.5)

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),axis=1))

[[0 0]
 [0 0]
 [0 1]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [19]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1531   76]
 [ 200  193]]


0.862

### There are 71 false positives & 200 True Negatives
### Also the accuracy of the model in unseen dataset is 86%